In [1]:
from IPython import get_ipython
ip = get_ipython()
ip.run_line_magic("reload_ext", "autoreload")  # these will enable module autoreloading
ip.run_line_magic("autoreload", "2")

### Get List of Reports Available

In [3]:
import os
from glob import glob
import ExportProvider.IBRK.Extract as ibrk
from lxml import etree
from ReportingStrategies.Slovenia.ReportGeneration import ReportGenerator
from ReportingStrategies.Slovenia.ReportGeneration import DividendReport
import ReportingStrategies.Slovenia.Mappers.IBRK as ibrkMapper
from Common.Configuration import TaxPayerInfo
from Common.Configuration import TaxPayerType
from Common.Configuration import ReportBaseConfig
from arrow import Arrow
import arrow

rootOfProject = os.getcwd()

reportsDirectory = os.path.join(rootOfProject, "BrokerExports")
generatedDirectory = os.path.join(rootOfProject, "GeneratedReports")

brokerExports = glob("*.xml", root_dir=reportsDirectory)
print(brokerExports)


def loadFileAndExtractLines(file: str):
    fileLocation = os.path.join(reportsDirectory, file)
    
    with open(fileLocation) as fobj:
        xml = fobj.read()

    root = etree.fromstring(xml)
    transactions = ibrk.extractCashTransactionFromXML(root)
    return transactions

brokerReports = list(map(loadFileAndExtractLines, brokerExports))
mergedReports = ibrk.mergeCashTransactions(brokerReports)

taxPayerInfo = TaxPayerInfo(
    taxNumber = "davcna",
    taxpayerType = TaxPayerType.PHYSICAL_SUBJECT,
    name = "ime priimek",
    address1 = "naslov",
    address2 = None,
    city = "mesto",
    postNumber = "postna #",
    postName = "posta",
    municipalityName = "obcina",
    birthDate = Arrow.now(),
    maticnaStevilka = "emsa",
    invalidskoPodjetje = False,
    resident = True,
    activityCode = "",
    activityName = "",
    countryID = "SI",
    countryName = "Slovenia"
)
basicReport = ReportGenerator(taxPayerInfo)
envelope = basicReport.createReportEnvelope()


reportconfig = ReportBaseConfig(fromDate=arrow.get("2022"), toDate=arrow.get("2023"))
dividendReport = DividendReport(reportConfig=reportconfig, taxPayerInfo=taxPayerInfo)

convertedCommonFormat = ibrkMapper.getExportGenericDividendLineFromCashTransactions(mergedReports)

csvReport = dividendReport.generateDataFrameReport(convertedCommonFormat)
csvReport.to_csv(os.path.join(generatedDirectory, 'export.csv'), index=False)

xmlReport = dividendReport.generateXmlReport(convertedCommonFormat, envelope)


etree.ElementTree(xmlReport).write(os.path.join(generatedDirectory, "Doh_Div_3.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)



['Tax_Relevant_Export_2022.xml']
Failed online lookup of ISIN (US9663875089), falling back to hardcoded company definitions
Failed online lookup of ISIN (US9663875089), falling back to hardcoded company definitions
